# **Práctica #1: Modelo para predicción de lluvia**

*Centro Universitario de Ciencias Exactas e Ingenierías*

*División de Tecnologías para la Integración Ciber-Humana*

*Ingeniería Biomédica*

<br>

*Mtra. Sofía Alejandra Aguilar Valdez*

2 de septiembre de 2022

## **Información del equipo**

```NOMBRES:``` \\
Victor Alvarado Aparicio \\
Dayana Analy Pacheco Bañuelos \\
Leonardo Juàrez Zucco

```CÓDIGOS:``` \\
215767685 \\
217535226 \\
220285869

```LINK REPOSITORIO:``` https://github.com/L-Rittmeister/U1_Team.git



## **Contenido**



1.   Resumen
2.   Marco teórico
3.   Objetivos
4.   Materiales y métodos
5.   Resultados
6.   Discusión
7.   Conclusiones
8.   Referencias



# **1. Resumen**
---
En este trabajo se pretendía realizar un modelo de predicción de la lluvia utilizando datos de la estación meteorológica del aeropuerto de Seattle, para este modelo teorizamos utilizar 4 entradas de información para las 4 variables con las que contábamos que eran la probabilidad de precipitación, las temperaturas máxima y mínima y una confirmación si llovió o no con una capa oculta. La metodología utilizada empieza primero con la descargar datos, después la construcción del modelo con las entradas y variables mencionadas antes, además declaramos la función de costo nn.CrossEntropyLoss(), y también un algoritmo de optimización optim.SGD(model.parameters(), lr=0.001), para el ajuste de parámetros de la red, por último el entrenamiento del modelo y la evaluación de este. Sin embargo, en la parte de la evaluación de la red aparentemente se logra hacer que funcione, pero no logramos revisar los resultados.

(100-300 palabras)

# **2. Marco teórico**
(300-800 palabras)

# **3. Objetivos**
---
Objetivo general y objetivos específicos
Procesar los datos y crear los objetos e instancias necesarios para poder hacer correr este modelo.
Crear un modelo de red neuronal que sea capaz de predecir si llovera o no en Seattle, Washington, USA, utilizando los datos de la estacion meteorologica del aeropuerto de Seattle.
Entrenar el modelo con los datos de entrenamiento y probarlo con los datos de prueba.
Evaluar el modelo y ver que tan bueno es.



(300-800 palabras)

# **4. Materiales y métodos**

## *Materiales*

Describir este conjunto de datos [[1]](https://www.kaggle.com/code/fatmakursun/rain-forecasting-with-artificial-neural-network/data).

## *Métodos*

1. Esquema de metodología
2. Descripción de los métodos y su implementación en código en forma de narrativa.

In [41]:
#Mounting drive storage to access data (if needed/running on collab then uncomment)
# Mount Google Drive
#from google.colab import drive
#drive.mount('/content/drive')

In [42]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error, confusion_matrix, classification_report
from sklearn.metrics import roc_curve
from tqdm.notebook import trange, tqdm
import time
import torch
from torch import nn
from torch.nn.functional import normalize
from torchvision import datasets, transforms
import torch.utils.data as data
from torch.utils.data import Dataset
import torch.nn.functional as F
from torch import optim
from torch.optim import SGD
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
from torch.nn.modules.activation import ReLU
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

In [43]:
#Pytorch and CUDA installation test
torch.randn(5).cuda()

tensor([ 1.3413, -0.8785,  0.5442, -1.5571, -2.9321], device='cuda:0')

In [44]:
#Load the dataset on a pandas dataframe
DataF = pd.read_csv("seattleWeather_1948-2017.csv")
DataF

,DATE,PRCP,TMAX,TMIN,RAIN
0,1948-01-01,0.47,51,42,True
1,1948-01-02,0.59,45,36,True
2,1948-01-03,0.42,45,35,True
3,1948-01-04,0.31,45,34,True
4,1948-01-05,0.17,45,32,True
...,...,...,...,...,...
25546,2017-12-10,0.00,49,34,False
25547,2017-12-11,0.00,49,29,False
25548,2017-12-12,0.00,46,32,False
25549,2017-12-13,0.00,48,34,False


In [45]:
#Drop NaN value
DataF.dropna(inplace=True)
DataF.reset_index(inplace=True, drop=True)

In [46]:
#eliminate Date column from the dataset
DataF.drop(['DATE'], axis=1, inplace=True)


In [47]:
#Transform the data to float
DataF['PRCP'] = DataF['PRCP'].astype(float)
DataF['TMAX'] = DataF['TMAX'].astype(float)
DataF['TMIN'] = DataF['TMIN'].astype(float)
DataF['RAIN'] = DataF['RAIN'].astype(float)


In [48]:
#Convert the dataframe to a numpy array
DataF = DataF.to_numpy()

In [49]:
#Create the dataset class using the PRCP, TMAX, TMIN as inputs and RAIN as label
class Dataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.len = data.shape[0]
        self.x_data = torch.from_numpy(self.data[:, :3])
        self.y_data = torch.from_numpy(self.data[:, 3])
        self.y_data = self.y_data.type(torch.LongTensor)
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    def __len__(self):
        return self.len


In [50]:
#Create the train and test dataset using the Dataset class and the data from the dataframe with 90% of the data for training and 10% for testing
train_data = Dataset(DataF[:int(0.9*len(DataF))])
test_data = Dataset(DataF[int(0.9*len(DataF)):])


In [51]:
#Create the train and test dataloader using the train and test dataset
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=True)


In [52]:
#Create the model class using 3 inputs and 2 hidden layers with 3 neurons each and 1 output
class Model(nn.Module):
    def __init__(self, n_inputs):
        super(Model, self).__init__()
        self.hidden1 = nn.Linear(n_inputs, 3)
        self.hidden2 = nn.Linear(3, 3)
        self.hidden3 = nn.Linear(3, 3)
        self.output = nn.Linear(3, 2)
        self.relu = nn.ReLU()
    def forward(self, x):
        x = self.relu(self.hidden1(x))
        x = self.relu(self.hidden2(x))
        x = self.relu(self.hidden3(x))
        x = self.output(x)
        return x


In [53]:
#Create the model using the model class and the number of inputs
model = Model(3).double().cuda()


In [54]:
#Create the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [55]:
#Define train function
def train(model, train_loader, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))


In [56]:
#Define test function
def test(model, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.cuda(), target.cuda()
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [57]:
#Train the model
train(model, train_loader, criterion, optimizer, 30)


Train Epoch: 0 [0/22993 (0%)]	Loss: 0.821950
Train Epoch: 0 [6400/22993 (28%)]	Loss: 0.702486
Train Epoch: 0 [12800/22993 (56%)]	Loss: 0.667476
Train Epoch: 0 [19200/22993 (83%)]	Loss: 0.648296
Train Epoch: 1 [0/22993 (0%)]	Loss: 0.605504
Train Epoch: 1 [6400/22993 (28%)]	Loss: 0.602824
Train Epoch: 1 [12800/22993 (56%)]	Loss: 0.585692
Train Epoch: 1 [19200/22993 (83%)]	Loss: 0.474671
Train Epoch: 2 [0/22993 (0%)]	Loss: 0.535307
Train Epoch: 2 [6400/22993 (28%)]	Loss: 0.561847
Train Epoch: 2 [12800/22993 (56%)]	Loss: 0.449493
Train Epoch: 2 [19200/22993 (83%)]	Loss: 0.548231
Train Epoch: 3 [0/22993 (0%)]	Loss: 0.494245
Train Epoch: 3 [6400/22993 (28%)]	Loss: 0.457933
Train Epoch: 3 [12800/22993 (56%)]	Loss: 0.477002
Train Epoch: 3 [19200/22993 (83%)]	Loss: 0.424483
Train Epoch: 4 [0/22993 (0%)]	Loss: 0.457653
Train Epoch: 4 [6400/22993 (28%)]	Loss: 0.432744
Train Epoch: 4 [12800/22993 (56%)]	Loss: 0.402286
Train Epoch: 4 [19200/22993 (83%)]	Loss: 0.417317
Train Epoch: 5 [0/22993 (0%)]	

In [58]:
#Test the model
test(model, test_loader, criterion)



Test set: Average loss: 0.0004, Accuracy: 2555/2555 (100%)



In [40]:
#Use the model to predict the rain
model.eval()
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.cuda(), target.cuda()
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)
        print(pred)


tensor([[1],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1]], device='cuda:0')
tensor([[1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1],
       

In [ ]:
#Save the model
torch.save(model.state_dict(), "model.pth")


# **5. Resultados**

# **6. Discusión**
Underfitting, overfitting, o no y el porqué
Comparación de su metodología con literatura asociada

# **7. Conclusiones**
En particular Este trabajo demoestró una complejidad y un desafio en la implementacion de redes neuronales en la plataforma pytorch y si bien los resultados no fueron satisfactorios fue una buena experiencia para aprender a utilizar esta plataforma y sus herramientas.
(100-300 palabras).

# **8. Referencias**

[1] Pb, V. (2020, February 18). Perceptron. Kaggle. Retrieved June 1, 2022, from https://www.kaggle.com/code/prashfio/perceptron/notebook